## Análise de Sentimentos – utilizando dados do Twitter 

### Importa todos os pacotes de Processamento de Linguagem Natural

In [1]:
import nltk
#nltk.download('all')

In [2]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

**Ler arquivo de dados e conta a quantidade de linhas**

In [3]:
# Dados coletados e Classificados pelo Data Scientist Rodrigo Santana Ferreira 
dataset = pd.read_csv('Tweets_Mg.csv',encoding='utf-8')

In [4]:
# número de tuplas
print("Número de linhas: ",len(dataset))

Número de linhas:  8199


In [7]:
# número de atributos
print("Número de atributos: ",len(dataset.columns))

Número de atributos:  26


In [8]:
# atributos
dataset.columns

Index(['Unnamed: 0', 'Created At', 'Text', 'Geo Coordinates.latitude',
       'Geo Coordinates.longitude', 'User Location', 'Username',
       'User Screen Name', 'Retweet Count', 'Classificacao', 'Observação',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [10]:
# tweets Já classificados - Positivos, Negativos e Neutros
dataset.head(8)

,Unnamed: 0,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,NaN,NaN,Brasil,Leonardo C Schneider,LeoCSchneider,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",-41.9333,-18.85,NaN,Wândell,klefnews,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,NaN,NaN,NaN,Ana estudando,estudandoconcur,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,NaN,NaN,NaN,Emily,Milly777,0,Negativo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Sat Jan 07 13:47:55 +0000 2017,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",NaN,NaN,Um Outro Mundo É Possível,Maristela Guimarães,florpimenta,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Wed Jan 04 23:00:53 +0000 2017,"""..E 25% dos mineiros dizem não torcer para ti...",NaN,NaN,NaN,Adriano,almanakfc,0,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Sun Jan 08 18:34:22 +0000 2017,"""A gigantesca barba do mal"" em destaque no cad...",NaN,NaN,"Belo Horizonte, Brasil",Editora Nemo,editoranemo,2,Neutro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# eliminando alguns atributos do dataset
atributos = ['Text', 'Classificacao', 'Created At', 'Geo Coordinates.latitude', 'Geo Coordinates.longitude',
             'User Location', 'User Screen Name',  ]
dataset2 = dataset.loc[:,atributos]
dataset = dataset2
dataset.head(8)

,Text,Classificacao,Created At,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,User Screen Name
0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro,Sun Jan 08 01:22:05 +0000 2017,NaN,NaN,Brasil,LeoCSchneider
1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro,Sun Jan 08 01:49:01 +0000 2017,-41.9333,-18.85,NaN,klefnews
2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro,Sun Jan 08 01:01:46 +0000 2017,-41.9333,-18.85,NaN,klefnews
3,��� https://t.co/BnDsO34qK0,Neutro,Wed Jan 04 21:43:51 +0000 2017,NaN,NaN,NaN,estudandoconcur
4,��� PSOL vai questionar aumento de vereadores ...,Negativo,Mon Jan 09 15:08:21 +0000 2017,NaN,NaN,NaN,Milly777
5,""" bom é bandido morto""\nDeputado Cabo Júlio é ...",Neutro,Sat Jan 07 13:47:55 +0000 2017,NaN,NaN,Um Outro Mundo É Possível,florpimenta
6,"""..E 25% dos mineiros dizem não torcer para ti...",Neutro,Wed Jan 04 23:00:53 +0000 2017,NaN,NaN,NaN,almanakfc
7,"""A gigantesca barba do mal"" em destaque no cad...",Neutro,Sun Jan 08 18:34:22 +0000 2017,NaN,NaN,"Belo Horizonte, Brasil",editoranemo


In [12]:
dataset.tail()

,Text,Classificacao,Created At,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,User Screen Name
8194,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 11:48:07 +0000 2017,NaN,NaN,NaN,lapiseirapentel
8195,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 12:10:19 +0000 2017,NaN,NaN,Belo Horizonte - Minas Gerais,Televans
8196,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 12:04:17 +0000 2017,NaN,NaN,Guarulhos - SP,leonardogru
8197,"Trio é preso suspeito de roubo, tráfico e abus...",Positivo,Thu Feb 09 12:10:04 +0000 2017,NaN,NaN,Brasil Natal/RN,parabolicalucas
8198,Trio suspeito de roubo de cargas é preso em Sa...,Positivo,Tue Feb 07 15:01:30 +0000 2017,NaN,NaN,Belo Horizonte - MG,tvrecordminas


**Conta a quantidade de linhas de tweets neutros, positivos e negativos**

In [13]:
len(dataset[dataset.Classificacao=='Neutro'])

2453

In [14]:
len(dataset[dataset.Classificacao=='Positivo'])

3300

In [15]:
len(dataset[dataset.Classificacao=='Negativo'])

2446

## Pre-Processamento 

** Separando tweets e suas classes
**

In [16]:
tweets = dataset['Text'].values
classes = dataset['Classificacao'].values

In [17]:
tweets[:10]

array([ '���⛪ @ Catedral de Santo Antônio - Governador Valadares/MG https://t.co/JSbKamIqUJ',
       '� @ Governador Valadares, Minas Gerais https://t.co/B3ThIDJCSf',
       '�� @ Governador Valadares, Minas Gerais https://t.co/dPkgzVR2Qw',
       '��� https://t.co/BnDsO34qK0',
       '��� PSOL vai questionar aumento de vereadores e prefeito de BH na Justiça - Politica - Estado de Minas https://t.co/DMg7BGsek5',
       '" bom é bandido morto"\nDeputado Cabo Júlio é condenado e fica inelegível por 10 anos - Politica - Estado de Minas https://t.co/3GfAqvrFHS',
       '"..E 25% dos mineiros dizem não torcer para time nenhum,mesmo dentro de um estado com Atlético-MG e Cruzeiro. Pq?.." https://t.co/fN5evlLQsR',
       '"A gigantesca barba do mal" em destaque no caderno Cultura do Estado de Minas. https://t.co/PxNurIkTlw',
       '"BB e governo de Minas travam disputa sobre depósitos judiciais" https://t.co/CnMu2A2Qo5',
       '"com vcs bh fica pequena!" Belo Horizonte (pron. [bɛloɾiˈzõntʃi]

In [18]:
classes[:10]

array(['Neutro', 'Neutro', 'Neutro', 'Neutro', 'Negativo', 'Neutro',
       'Neutro', 'Neutro', 'Negativo', 'Neutro'], dtype=object)

**Gerando o modelo**

In [19]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(tweets)
modelo = MultinomialNB()
modelo.fit(freq_tweets,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

** Testando o modelo com algumas instâncias simples**

In [20]:
testes = ['Esse governo está no início, vamos ver o que vai dar',
          'Estou muito feliz com o governo de Minas esse ano',
          'O estado de Minas Gerais decretou calamidade financeira!!!',
          'A segurança desse país está deixando a desejar',
          'O governador de Minas é do PT']

In [21]:
freq_testes = vectorizer.transform(testes)

In [22]:
freq_testes.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [23]:
# Fazendo a classificação com o modelo treinado.
modelo.predict(freq_testes)

array(['Neutro', 'Neutro', 'Negativo', 'Negativo', 'Neutro'], 
      dtype='<U8')

In [31]:
df = pd.DataFrame()
df['tweet'] = testes
df['classificacao'] = modelo.predict(freq_testes)

tweets = testes
classificacao = modelo.predict(freq_testes)

print ("Novas Instancias Classificadas...")
for i in range(len(df)):
    print (classificacao[i], " -> ", tweets[i] )

Novas Instancias Classificadas...
Neutro  ->  Esse governo está no início, vamos ver o que vai dar
Neutro  ->  Estou muito feliz com o governo de Minas esse ano
Negativo  ->  O estado de Minas Gerais decretou calamidade financeira!!!
Negativo  ->  A segurança desse país está deixando a desejar
Neutro  ->  O governador de Minas é do PT


** Avaliando o modelo **

In [32]:
# Fazendo o cross validation do modelo
resultados = cross_val_predict(modelo, freq_tweets, classes, cv=10)

In [33]:
resultados[:10]

array(['Neutro', 'Neutro', 'Neutro', 'Positivo', 'Neutro', 'Neutro',
       'Neutro', 'Neutro', 'Negativo', 'Neutro'], 
      dtype='<U8')

In [34]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(classes,resultados)

0.8831564824978656

### Matriz de Confusão

In [38]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['REAL'], colnames=['PREDITO'], margins=True))

PREDITO   Negativo  Neutro  Positivo   All
REAL                                      
Negativo      2275     162         9  2446
Neutro         240    2067       146  2453
Positivo        45     356      2899  3300
All           2560    2585      3054  8199


In [37]:
# Medidas de validação do modelo
sentimento=['Positivo','Negativo','Neutro']
print (metrics.classification_report(classes,resultados,sentimento))

             precision    recall  f1-score   support

   Positivo       0.95      0.88      0.91      3300
   Negativo       0.89      0.93      0.91      2446
     Neutro       0.80      0.84      0.82      2453

avg / total       0.89      0.88      0.88      8199

